# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Y. Wang  ->  Y. Wang  |  ['Y. Wang']
M. Zhang  ->  M. Zhang  |  ['M. Zhang']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Arxiv has 83 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2507.07412
extracting tarball to tmp_2507.07412... done.
Retrieving document from  https://arxiv.org/e-print/2507.07629


extracting tarball to tmp_2507.07629...

 done.


Found 132 bibliographic references in tmp_2507.07629/ERO11_WL_arxiv.bbl.
Issues with the citations
syntax error in line 242: '=' expected
Retrieving document from  https://arxiv.org/e-print/2507.07798
extracting tarball to tmp_2507.07798... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.07629-b31b1b.svg)](https://arxiv.org/abs/2507.07629) | **Euclid: Early Release Observations. Weak gravitational lensing analysis of Abell 2390**  |
|| T. Schrabback, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-07-11*|
|*Comments*| *This paper is published on behalf of the Euclid Consortium. 28 pages, 27 figures, 7 tables. Submitted to A&A*|
|**Abstract**|            The Euclid space telescope of the European Space Agency (ESA) is designed to provide sensitive and accurate measurements of weak gravitational lensing distortions over wide areas on the sky. Here we present a weak gravitational lensing analysis of early Euclid observations obtained for the field around the massive galaxy cluster Abell 2390 as part of the Euclid Early Release Observations programme. We conduct galaxy shape measurements using three independent algorithms (LensMC, KSB+, and SourceXtractor++). Incorporating multi-band photometry from Euclid and Subaru/Suprime-Cam, we estimate photometric redshifts to preferentially select background sources from tomographic redshift bins, for which we calibrate the redshift distributions using the self-organising map approach and data from the Cosmic Evolution Survey (COSMOS). We quantify the residual cluster member contamination and correct for it in bins of photometric redshift and magnitude using their source density profiles, including corrections for source obscuration and magnification. We reconstruct the cluster mass distribution and jointly fit the tangential reduced shear profiles of the different tomographic bins with spherical Navarro--Frenk--White profile predictions to constrain the cluster mass, finding consistent results for the three shape catalogues and good agreement with earlier measurements. As an important validation test we compare these joint constraints to mass measurements obtained individually for the different tomographic bins, finding good consistency. More detailed constraints on the cluster properties are presented in a companion paper that additionally incorporates strong lensing measurements. Our analysis provides a first demonstration of the outstanding capabilities of Euclid for tomographic weak lensing measurements.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.07412-b31b1b.svg)](https://arxiv.org/abs/2507.07412) | **Imaging-Spectroscopic diagnosis of the Giant Sloshing Spiral in the Virgo Cluster with the Einstein Probe Follow-up X-ray Telescope**  |
|| X. Zheng, et al. -- incl., <mark>Y. Wang</mark>, <mark>M. Zhang</mark> |
|*Appeared on*| *2025-07-11*|
|*Comments*| *11 pages, 16 figures*|
|**Abstract**|            We performed deep X-ray observations of the Virgo cluster using the Einstein Probe Follow-up X-ray Telescope (EP-FXT) with a total exposure of 295 ks. Leveraging the large field of view (FoV) and low particle background of EP-FXT, the image reveals a giant spiral feature connecting the cold fronts in the northwest and southeast, forming a coherent structure consistent with earlier results from XMM-Newton and Suzaku. We also present two-dimensional maps of temperature, metallicity, and entropy across the Virgo Cluster, covering a FoV of approximately 28.5 arcmin. These maps clearly show a spiral structure with high density, low temperature, high metallicity, and low entropy. The results support a scenario where the spiral morphology arises from gas sloshing driven by a minor merger. Additionally, EP-FXT temperature measurements agree well with XMM-Newton data within uncertainties.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.07798-b31b1b.svg)](https://arxiv.org/abs/2507.07798) | **Exploring non-cold dark matter in a scenario of dynamical dark energy with DESI DR2 data**  |
|| T.-N. Li, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-07-11*|
|*Comments*| *8 pages, 5 figures*|
|**Abstract**|            Recent observations of DESI hint that dark matter (DM) may not be cold but have a non-zero equation of state (EoS) parameter, and that dark energy (DE) may not be a cosmological constant. In this work, we explore the possibility of a non-zero DM EoS parameter within the framework of dynamical DE. We perform analysis by using the latest baryon acoustic oscillation (BAO) data from DESI DR2, the cosmic microwave background (CMB) data from Planck, and the type Ia supernova (SN) data from DESY5 and PantheonPlus. When using the combination of CMB, BAO, and SN data, our results indicate a preference for a non-zero DM EoS parameter at the $2.8\sigma$ and $3.3\sigma$ level within the content of a constant DE EoS. In contrast, for a time-evolving DE EoS parameterized by $w_0$ and $w_a$, this preference decreases to $0.8\sigma$ and $1.1\sigma$. Furthermore, allowing a non-zero DM EoS yields best-fit values of $w_0$ and $w_a$ that exhibit smaller deviations from the $\Lambda$CDM expectations, and Bayesian evidence analysis shows a comparable preference for this model relative to $\Lambda$CDM. The overall results of this work indicate that a non-zero DM EoS parameter warrants further exploration and investigation.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2507.07629/./figs/joint_maghisto_unweighted.png', 'tmp_2507.07629/./figs/image_kappa_KSB.jpg', 'tmp_2507.07629/./figs/nz_mag22-24p5_6bins_ser++_number-1.png', 'tmp_2507.07629/./figs/nz_mag24p5-26p5_6bins_ser++_number-1.png']
copying  tmp_2507.07629/./figs/joint_maghisto_unweighted.png to _build/html/
copying  tmp_2507.07629/./figs/image_kappa_KSB.jpg to _build/html/
copying  tmp_2507.07629/./figs/nz_mag22-24p5_6bins_ser++_number-1.png to _build/html/
copying  tmp_2507.07629/./figs/nz_mag24p5-26p5_6bins_ser++_number-1.png to _build/html/
exported in  _build/html/2507.07629.md
    + _build/html/tmp_2507.07629/./figs/joint_maghisto_unweighted.png
    + _build/html/tmp_2507.07629/./figs/image_kappa_KSB.jpg
    + _build/html/tmp_2507.07629/./figs/nz_mag22-24p5_6bins_ser++_number-1.png
    + _build/html/tmp_2507.07629/./figs/nz_mag24p5-26p5_6bins_ser++_number-1.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\mat}[1]{\tens{#1}}$
$\newcommand{\comm}[1]{\textcolor{red}{#1}}$
$\newcommand{\orcid}[1]$
$\newcommand{\pdr}{PDR}$</div>



<div id="title">

# $\Euclid$: Early Release Observations. Weak gravitational lensing analysis of Abell 2390$\thanks{This paper is published on behalf of the Euclid Consortium.}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.07629-b31b1b.svg)](https://arxiv.org/abs/2507.07629)<mark>Appeared on: 2025-07-11</mark> -  _This paper is published on behalf of the Euclid Consortium. 28 pages, 27 figures, 7 tables. Submitted to A&A_

</div>
<div id="authors">

T. Schrabback, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** The $\Euclid$ space telescope of the  European Space Agency (ESA) is designed to provide sensitive and accurate measurements of weak gravitational lensing distortions over wide areas on the sky.   Here we present a weak gravitational lensing analysis of early $\Euclid$ observations obtained for the field around the massive galaxy cluster Abell 2390 as part of the $\Euclid$ Early Release Observations programme.   We conduct   galaxy shape measurements   using three independent   algorithms ( \texttt{LensMC} , \texttt{KSB+} , and \texttt{SourceXtractor++} ).  Incorporating multi-band photometry from $\Euclid$ and Subaru/Suprime-Cam,  we estimate photometric  redshifts to preferentially select background sources from tomographic redshift bins,  for which we calibrate the redshift distributions using the self-organising map approach and  data from the Cosmic Evolution Survey (COSMOS).  We quantify the residual cluster member contamination and correct for it in bins of photometric redshift and magnitude using their source density profiles, including corrections for source obscuration and magnification. We reconstruct the cluster mass distribution and jointly fit the tangential reduced shear profiles of the different tomographic binswith spherical Navarro--Frenk--White profile predictions to constrain the cluster mass, finding consistent results for the three shape catalogues and good agreement with earlier measurements.        As an important validation test       we compare these joint constraints to mass measurements obtained individually for the different tomographic bins,finding good consistency.    More detailed    constraints on the cluster properties are presented in a companion paper    that additionally incorporates strong lensing measurements.    Our analysis provides a first demonstration of the outstanding capabilities of $\Euclid$ for tomographic  weak lensing measurements.

</div>

<div id="div_fig1">

<img src="tmp_2507.07629/./figs/joint_maghisto_unweighted.png" alt="Fig6" width="100%"/>

**Figure 6. -** Number density of objects in the \texttt{LensMC}(dashed),  \texttt{SE++}(solid), and \texttt{KSB+}(dotted) weak lensing source catalogues, computed
within the central
$0\fdg555 \times 0\fdg555$
of the VIS stack, after applying shear selection cuts and removal of objects in masked areas, but without applying photometric redshift selections. (*fig:count_joint*)

</div>
<div id="div_fig2">

<img src="tmp_2507.07629/./figs/image_kappa_KSB.jpg" alt="Fig26" width="100%"/>

**Figure 26. -** As Fig.\thinspace\ref{fig:image_mass_overlay_lensMC}, but employing the \texttt{KSB+} shear catalogue.  (*fig:image_mass_overlay_KSB*)

</div>
<div id="div_fig3">

<img src="tmp_2507.07629/./figs/nz_mag22-24p5_6bins_ser++_number-1.png" alt="Fig5.1" width="50%"/><img src="tmp_2507.07629/./figs/nz_mag24p5-26p5_6bins_ser++_number-1.png" alt="Fig5.2" width="50%"/>

**Figure 5. -** Redshift distributions for each of our redshift and magnitude bins using the \texttt{SE++} shear weights. The upper panel shows the distributions for the magnitude range $22<\IE<24.5$, and the lower panel for the range $24.5<\IE<26.5$. The black curve shows the geometric lensing efficiency $\beta(z)$, with the corresponding axis plotted on the right. (*fig:nz*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.07629"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

139  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

15  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
